#CIS5190 Final Project ---- Stock Price Prediction
By Xinyu Cai, Wanqing Ding and Leila Zhu<br>
April 22nd, 2023

In [ ]:
import pandas as pd
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
from itertools import cycle
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})
%matplotlib inline

# For time stamps
from datetime import datetime

# Model import
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

#ARIMA Model
import statsmodels.graphics.tsaplots as sgt
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

#**I. Stock Price Data**
[Link to CSV](https://www.kaggle.com/code/ysthehurricane/advanced-stock-pred-using-svr-rfr-knn-lstm-gru/input)

## 1.1 Data Loading and Basic Preprocessing

In [ ]:
#unhashtag to install PyDrive install once
#!pip install -U -q PyDrive

#import dataset from GoogleDrive set up
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#please authenticate.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#files should be accessible using berkeley.edu
stock_link = 'https://drive.google.com/file/d/1bKGOTVhOuAAv7D1iD0F2lOkm7YrRMOJU/view?usp=share_link'
downloaded = drive.CreateFile({'id':"1bKGOTVhOuAAv7D1iD0F2lOkm7YrRMOJU"}) 
downloaded.GetContentFile('RELIANCE.csv')  
df = pd.read_csv('RELIANCE.csv')

The dataset consists of historical stock prices for a particular company. The data covers a period from 08/19/2020 to 08/18/2021 and includes information on the opening, closing, high, and low prices for the stock, as well as the volume of shares traded on each day. Additionally, the dataset includes information on the adjusted close price, which takes into account any dividends or stock splits that may have occurred during the time period. 

In [ ]:
# sneak peak at the data
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.Date.min()

In [ ]:
df.Date.max()

## 1.2 Data Preprocessing

Checking Null Values

In [ ]:
# Checking null value
df.isnull().sum()

Looking at the info table and checking null values above, we can see that all columns have 1 row of `NaN` values except the Date column. In fact, there are only 1 row of missing value in the entire dataframe! We are going to drop this one row, so that information we kept is still complete and essential. We procede by dropping the missing value rows.

In [ ]:
df.dropna(inplace=True)
df.isna().any()

Convert Date from String to Date format in order for the modeling part.

In [ ]:
# convert date field from string to Date format and make it index
df['Date'] = pd.to_datetime(df.Date)
df.head()

#**II. EDA**

### 2.1 Data Visualization

#### Close Price

The closing price of a stock is the final price at which the stock is traded during a specific trading day. It represents the market's consensus value of a stock at the end of the trading session. Investors and financial analysts often use the closing price as a reference point for assessing the stock's performance over time, comparing it to previous closing prices to analyze trends and make investment decisions.

#### Open Price

Open price for a stock refers to the initial price at which a share of a particular company begins trading when the stock market opens for the day. The open price is influenced by various factors, including pre-market trading, the previous day's closing price, overall market sentiment, news and events related to the company or the broader market, and the supply and demand for the stock.

#### High Price

The high price for a stock refers to the highest price at which a share of a particular company has traded during a specific period, such as a trading day, week, month, or year. The high price represents the peak value that investors were willing to pay for the stock during that time frame. 

#### Low Price

The low price for a stock refers to the lowest price at which a share of a particular company has traded during a specific period, such as a trading day, week, month, or year. The low price represents the minimum value that investors were willing to pay for the stock during that time frame. This information is often used by traders and investors to analyze price trends, as well as to assess the stock's historical performance and potential support levels in technical analysis. Similar to the high price, the low price is not static and will change as new low prices are reached.

In [ ]:
plt.figure(figsize=(15, 12))
column = ['Open', 'Close', 'High', 'Low']
for i in range(4):
  plt.subplot(2, 2, i+1)
  plt.plot(df.Date, df[column[i]])
  plt.ylabel('Close price')
  plt.grid(True)
  plt.title(str(column[i]))

### 2.2 Moving Average

The moving average (MA) is a simple technical analysis tool that smooths out price data by creating a constantly updated average price. The average is taken over a specific period of time, like 10 days, 20 minutes, 30 weeks, or any time period the trader chooses.

In [ ]:
ma_day = [10, 20, 50]

for ma in ma_day:
      column_name = f"MA for {ma} days"
      df[column_name] = df['Close'].rolling(ma).mean()

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(df.Date, df[['Close', 'MA for 10 days', 'MA for 20 days', 'MA for 50 days']])
plt.legend(['Close', 'MA for 10 days', 'MA for 20 days', 'MA for 50 days'])
plt.ylabel('Close Price')
plt.xlabel('Date')
plt.grid(True)
plt.title('Moving Average for Close Price')

### 2.3 Daily Return

We're now going to analyze the risk of the stock. In order to do so we'll need to take a closer look at the daily changes of the stock, and not just its absolute value. Let's go ahead and use pandas to retrieve teh daily returns for the stock.

In [ ]:
df['Daily Return'] = df['Close'].pct_change()
plt.figure(figsize=(10, 8))
plt.plot(df.Date, df['Daily Return'])
plt.ylabel('Percentage Change')
plt.xlabel('Date')
plt.grid(True)
plt.title('Daily Return (% Change)')

#**III. Modeling**

## 3.0 Data Preparation Before Model

In [ ]:
df_model = df[["Open", "High", "Low", "Volume", "Close"]]

In [ ]:
training_size=int(len(df_model)*0.07)
train_df = df_model.loc[0:training_size,]
test_df = df_model.loc[training_size:, ]
X_train = train_df[["Open", "High", "Low", "Volume"]]
y_train = train_df["Close"]
X_test = test_df[["Open", "High", "Low", "Volume"]]
y_test = test_df["Close"]

## 3.1 OLS

### 3.1.1 Model

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
y_train_pred_lr = lr.predict(X_train)
y_test_pred_lr = lr.predict(X_test)

### 3.1.2 Evaluation

**Evaluation metrices RMSE, MSE and MAE**

Root Mean Square Error (RMSE), Mean Square Error (MSE) and Mean absolute Error (MAE) are a standard way to measure the error of a model in predicting quantitative data.

In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(y_train,y_train_pred_lr)))
print("Train data MSE: ", mean_squared_error(y_train,y_train_pred_lr))
print("Train data MAE: ", mean_absolute_error(y_train,y_train_pred_lr))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(y_test,y_test_pred_lr)))
print("Test data MSE: ", mean_squared_error(y_test,y_test_pred_lr))
print("Test data MAE: ", mean_absolute_error(y_test,y_test_pred_lr))

The difference between the train and test data metrics is not significant, which is a good sign. This indicates that the model is not overfitting, as the performance on the test set is similar to that on the training set.

**Explained Variance Score**


Explained variance regression score is an evaluation metric used in regression tasks to measure the proportion of the total variance in the target variable that is explained by the predictive model. It is a useful metric for understanding how well the model captures the underlying structure of the data. It ranges from 0 to 1, with 1 being the best possible score. A score of 1 indicates that the model perfectly explains the variance in the target variable, while a score of 0 indicates that the model does not explain any variance in the target variable.

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(y_train, y_train_pred_lr))
print("Test data explained variance regression score:", explained_variance_score(y_test, y_test_pred_lr))

The explained variance regression score is quite high for both the train and test data, which indicates that the model maybe is doing a good job of capturing the variance in the target variable (stock price).

**R<sup>2</sup> score**

The R-squared (R2) score, also known as the coefficient of determination, is a metric used to evaluate the performance of a regression model. It measures the proportion of the variance in the dependent variable (target) that can be explained by the independent variables (predictors) in the model.

The R2 score ranges from 0 to 1, with higher values indicating a better fit of the model to the data. An R2 score of 1 indicates that the model explains 100% of the variance in the target variable, while an R2 score of 0 means that the model does not explain any variance in the target variable.

In [ ]:
print("Train data R2 score:", r2_score(y_train, y_train_pred_lr))
print("Test data R2 score:", r2_score(y_test, y_test_pred_lr))

The R-squared score, which measures the proportion of the total variance in the target variable explained by the model, is also high for both the train and test data. This means that the model is accounting for a large portion of the variability in the stock prices.

**Prediction Visualization**

We draw a plot to compare between the original stock close price and predicted close price for test dataset.

In [ ]:
df_plot_lr = df[["Date"]]
df_plot_lr = df_plot_lr.loc[training_size:, ]
df_plot_lr["y_test"] = y_test
df_plot_lr["y_test_pred_lr"] = y_test_pred_lr

In [ ]:
# Plot the actual and predicted close prices
plt.plot(df_plot_lr['Date'], df_plot_lr['y_test'], label='Actual Close Price', color='blue')
plt.plot(df_plot_lr['Date'], df_plot_lr['y_test_pred_lr'], label='Predicted Close Price', color='orange')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.title('Actual vs Predicted Close Prices')
plt.legend()
plt.show()

### 3.1.3 Summarization

We see that linear regression gives a very good result overall. **However**, this result is not reliable. Here are some of the key reasons why linear regression might not be a good model for predicting stock prices:

1. **Linearity assumption**: Linear regression assumes that the relationship between the independent and dependent variables is linear. However, stock prices are influenced by a multitude of factors, including market sentiment, economic indicators, and company-specific news, which can cause nonlinear relationships between the predictors and stock prices.

2. **Independence of observations**: Linear regression assumes that the observations are independent of each other. In the case of stock prices, this is not true, as prices are influenced by past prices and market trends, resulting in a time series data structure with inherent autocorrelation.

3. **Constant variance (homoscedasticity)**: Linear regression assumes that the variance of the error terms is constant across all levels of the independent variables. Stock prices are known to exhibit heteroscedasticity, with periods of high volatility followed by periods of low volatility.

4. **Normal distribution of errors**: Linear regression assumes that the error terms follow a normal distribution. However, stock prices tend to exhibit non-normal distributions, with heavy tails (kurtosis) and skewness, which violates this assumption.

## 3.2 Support Vector Regression (SVR)

Support Vector Regression (SVR) is the combination of a Support Vector Machines and Regression.

Why we choose **Support Vector Regressor**?
* Linear Regression does not work for our data because our data has many fluctuations and a linear line of best fit would give poor predictions on stock data. A SVM will not work on our data because we aren’t classifying between two different classes.
* With stock data, we are not predicting a class, we are predicting the next value in a series.
* Using regression we try to minimize the cost function using something like gradient descent. With SVMs we try to try to draw a hyperplane between 2 different classes. So SVR is the combo of the 2, we try to minimize the error within a certain threshold.

### 3.2.1 Data Preparation for SVR

We are going to predict the stock price using Support Vector Regression. In order to let the model learn more efficiently, we will treat this as a time series question. In other words, our sole x feature will be the close price. Thus, we will redo the data preparation in this section.

**Normalize Close Price**

Normalizing the close price of a stock before implementing a model to predict stock prices is important for several reasons:

1. Scale uniformity: Financial data can have a wide range of values, which can lead to inconsistencies in scale when comparing different stocks or other financial instruments. Normalization helps to ensure that all data points are on a similar scale, allowing the model to process and compare them more effectively.

2. Improved convergence: Machine learning models, particularly neural networks, often perform better and converge faster when their input features have similar scales. Normalizing the data helps to prevent any single feature from dominating the learning process, which could lead to suboptimal results or slow convergence.

3. Comparability: When working with time series, normalizing the close prices enables easier comparison between them. This is particularly useful when you want to analyze or predict trends across different stocks or market sectors.

4. Mitigating the impact of outliers: Extreme values or outliers in the data can disproportionately affect the model's predictions. By normalizing the close prices, the influence of outliers is reduced, leading to more robust and accurate predictions.

In [ ]:
# Select the close price column (replace 'close' with the actual column name in your dataset)
close_price = df['Close']

# Reshape the close price data
close_price = np.array(close_price).reshape(-1, 1)

# Initialize the MinMaxScaler with the desired feature range
scaler = MinMaxScaler(feature_range=(0, 1))

# Fit and transform the close price data
normalized_close_price = scaler.fit_transform(close_price)

**Split into train and test data set**

We can't use train_test_split to split the data in the time series model. Instead, we can only split the data sequentially. We use 80% of the data for training, and 20% of the data for testing.

In [ ]:
training_size=int(len(close_price)*0.80)
train_df = normalized_close_price[0:training_size,]
test_df = normalized_close_price[training_size:, ]
print("train_df: ", train_df.shape)
print("test_df: ", test_df.shape)

**Transform Close price base on Time-series-analysis forecasting requirement**

Now, we are going to create x and label features for the model. We will create in this way:

**Example**

Suppose we have a following sequence of time series data:

Sequence: 367, 363, 364, 368, 371, 372, 369, 367, 366

Timestamp = 3

\begin{array}{|c|c|} \hline
Train Data & Label\\ \hline
367, 363, 364 & 368 \\ \hline
363, 364, 368 & 371 \\\hline
364, 368, 371 & 372 \\ \hline
368, 371, 372 & 369  \\ \hline
371, 372, 369 & 367  \\ \hline
372, 369, 367 & 366 \\ \hline
\end{array}

In [ ]:
def create_dataset(dataset, time_step=1):
    dataX = [dataset[i:(i + time_step), 0] for i in range(len(dataset) - time_step - 1)]
    dataY = [dataset[i + time_step, 0] for i in range(len(dataset) - time_step - 1)]
    
    return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 10
X_train_ts, y_train_ts = create_dataset(train_df, time_step)
X_test_ts, y_test_ts = create_dataset(test_df, time_step)

print("X_train_ts: ", X_train_ts.shape)
print("y_train_ts: ", y_train_ts.shape)
print("X_test_ts: ", X_test_ts.shape)
print("y_test_ts", y_test_ts.shape)

### 3.2.2 Base Line Model

In [ ]:
svr_rbf = SVR(kernel= 'rbf', C= 1e2, gamma= 0.1)
svr_rbf.fit(X_train_ts, y_train_ts)

In [ ]:
y_train_pred_svr_rbf=svr_rbf.predict(X_train_ts)
y_test_pred_svr_rbf=svr_rbf.predict(X_test_ts)

In [ ]:
# Transform back to original form
y_train_pred_svr_rbf = scaler.inverse_transform(y_train_pred_svr_rbf.reshape(-1,1))
y_test_pred_svr_rbf = scaler.inverse_transform(y_test_pred_svr_rbf.reshape(-1,1))
original_y_train = scaler.inverse_transform(y_train_ts.reshape(-1,1)) 
original_y_test = scaler.inverse_transform(y_test_ts.reshape(-1,1)) 

#### 3.2.2.1 Evaluation (Base Line Model)

**Evaluation metrices RMSE, MSE and MAE**

In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_y_train,y_train_pred_svr_rbf)))
print("Train data MSE: ", mean_squared_error(original_y_train,y_train_pred_svr_rbf))
print("Train data MAE: ", mean_absolute_error(original_y_train,y_train_pred_svr_rbf))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_y_test,y_test_pred_svr_rbf)))
print("Test data MSE: ", mean_squared_error(original_y_test,y_test_pred_svr_rbf))
print("Test data MAE: ", mean_absolute_error(original_y_test,y_test_pred_svr_rbf))

**Explained Variance Score**

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_y_train, y_train_pred_svr_rbf))
print("Test data explained variance regression score:", explained_variance_score(original_y_test, y_test_pred_svr_rbf))

**R<sup>2</sup> score**

In [ ]:
print("Train data R2 score:", r2_score(original_y_train, y_train_pred_svr_rbf))
print("Test data R2 score:", r2_score(original_y_test, y_test_pred_svr_rbf))

**Prediction Visualization**

The code below was based on the original code of this [Kaggle notenook](https://www.kaggle.com/code/ysthehurricane/advanced-stock-pred-using-svr-rfr-knn-lstm-gru/notebook). However, the code below was improved based on the original one and was organized using function.

In [ ]:
def prepare_predictions_plot(closedf, train_predict, test_predict, time_step):
    look_back = time_step
    trainPredictPlot = np.empty_like(closedf)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[look_back:len(train_predict) + look_back, :] = train_predict

    testPredictPlot = np.empty_like(closedf)
    testPredictPlot[:, :] = np.nan
    testPredictPlot[len(train_predict) + (look_back * 2) + 1:len(closedf) - 1, :] = test_predict

    return trainPredictPlot, testPredictPlot

def plot_stock_prices(close_stock, closedf, trainPredictPlot, testPredictPlot):
    names = cycle(['Original close price', 'Train predicted close price', 'Test predicted close price'])

    plotdf = pd.DataFrame({'date': close_stock['Date'],
                           'original_close': close_stock['Close'],
                           'train_predicted_close': trainPredictPlot.reshape(1, -1)[0].tolist(),
                           'test_predicted_close': testPredictPlot.reshape(1, -1)[0].tolist()})

    fig = px.line(plotdf, x='date', y=['original_close', 'train_predicted_close', 'test_predicted_close'],
                  labels={'value': 'Stock price', 'date': 'Date'})
    fig.update_layout(title_text='Comparison between Close Price vs Predicted Close Price',
                      plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
    fig.for_each_trace(lambda t: t.update(name=next(names)))

    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    fig.show()

close_stock = df[["Date", "Close"]]
trainPredictPlot, testPredictPlot = prepare_predictions_plot(close_price, y_train_pred_svr_rbf, y_test_pred_svr_rbf, 10)
plot_stock_prices(close_stock, close_price, trainPredictPlot, testPredictPlot)

### 3.2.3 Hyperparameter Tuning: SVR with Grid Search

SVR has several hyperparameters that control its behavior and performance. We will tune 3 parameters in the following section:

1. **Kernel**: The kernel function used to transform the input data into a higher-dimensional space. Common choices for the kernel function include 'linear', 'poly', 'rbf' (Radial basis function), and 'sigmoid'. We will fix the kernel to be 'rbf'.

2. **C (Cost or Regularization parameter)**: This parameter controls the trade-off between achieving a low training error and a low testing error, i.e., the balance between overfitting and underfitting. A smaller value of C creates a wider margin, which may result in more training errors but better generalization to the test data. A larger value of C creates a narrower margin, which minimizes training errors at the potential cost of poorer generalization.

3. **epsilon**: This parameter defines the 'insensitive zone' within which no penalty is associated with errors. It controls the width of the tube around the regression line, inside which errors are not penalized. A larger epsilon value results in a smoother regression function, while a smaller epsilon value results in a more flexible function that may be more prone to overfitting.

4. **gamma**: It controls the shape of the decision boundary. A small gamma value will produce a more flexible decision boundary, while a large gamma value will produce a more rigid boundary. 

In [ ]:
paramters = {
        'C':[0.001,0.01,0.1,1,100],
        'epsilon': [0.001, 0.01, 0.1, 1, 10, 100],
        'gamma': [0.01, 0.1, 1, 5, 8, 40]
    }

In [ ]:
gsc = GridSearchCV(
        estimator=SVR(kernel='rbf'),
        param_grid=paramters,
        cv=10,
        scoring='neg_mean_squared_error'
    )

In [ ]:
grid_result = gsc.fit(X_train_ts,y_train_ts)

In [ ]:
best_param = grid_result.best_params_
svr_model = SVR(kernel='rbf', C=best_param['C'], epsilon=best_param['epsilon'], gamma=best_param['gamma'])
svr_model.fit(X_train_ts, y_train_ts)
y_train_pred_svr_rbf_cv=svr_model.predict(X_train_ts)
y_test_pred_svr_rbf_cv=svr_model.predict(X_test_ts)

In [ ]:
# Transform back to original form
y_train_pred_svr_rbf_cv = scaler.inverse_transform(y_train_pred_svr_rbf_cv.reshape(-1,1))
y_test_pred_svr_rbf_cv = scaler.inverse_transform(y_test_pred_svr_rbf_cv.reshape(-1,1))
original_y_train = scaler.inverse_transform(y_train_ts.reshape(-1,1)) 
original_y_test = scaler.inverse_transform(y_test_ts.reshape(-1,1)) 

#### 3.2.3.1 Evaluation (Tuned Model)

In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_y_train,y_train_pred_svr_rbf_cv)))
print("Train data MSE: ", mean_squared_error(original_y_train,y_train_pred_svr_rbf_cv))
print("Train data MAE: ", mean_absolute_error(original_y_train,y_train_pred_svr_rbf_cv))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_y_test,y_test_pred_svr_rbf_cv)))
print("Test data MSE: ", mean_squared_error(original_y_test,y_test_pred_svr_rbf_cv))
print("Test data MAE: ", mean_absolute_error(original_y_test,y_test_pred_svr_rbf_cv))

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_y_train, y_train_pred_svr_rbf_cv))
print("Test data explained variance regression score:", explained_variance_score(original_y_test, y_test_pred_svr_rbf_cv))

In [ ]:
print("Train data R2 score:", r2_score(original_y_train, y_train_pred_svr_rbf_cv))
print("Test data R2 score:", r2_score(original_y_test, y_test_pred_svr_rbf_cv))

In [ ]:
trainPredictPlot, testPredictPlot = prepare_predictions_plot(close_price, y_train_pred_svr_rbf_cv, y_test_pred_svr_rbf_cv, 10)
plot_stock_prices(close_stock, close_price, trainPredictPlot, testPredictPlot)

### 3.2.5 SVR with Bagging

In [ ]:
bagging_svr = BaggingRegressor(base_estimator = SVR(kernel='rbf', C=best_param['C'], epsilon=best_param['epsilon'], gamma=best_param['gamma']),n_estimators = 1000, random_state=42)
model = bagging_svr.fit(X_train_ts, y_train_ts)
y_train_pred_bag_svr = model.predict(X_train_ts)
y_test_pred_bag_svr = model.predict(X_test_ts)

In [ ]:
# Transform back to original form
y_train_pred_bag_svr = scaler.inverse_transform(y_train_pred_bag_svr.reshape(-1,1))
y_test_pred_bag_svr = scaler.inverse_transform(y_test_pred_bag_svr.reshape(-1,1))
original_y_train = scaler.inverse_transform(y_train_ts.reshape(-1,1)) 
original_y_test = scaler.inverse_transform(y_test_ts.reshape(-1,1))

#### 3.2.5.1 Evaluation (Bagging SVR)

In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_y_train,y_train_pred_bag_svr)))
print("Train data MSE: ", mean_squared_error(original_y_train,y_train_pred_bag_svr))
print("Train data MAE: ", mean_absolute_error(original_y_train,y_train_pred_bag_svr))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_y_test,y_test_pred_bag_svr)))
print("Test data MSE: ", mean_squared_error(original_y_test,y_test_pred_bag_svr))
print("Test data MAE: ", mean_absolute_error(original_y_test,y_test_pred_bag_svr))

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_y_train, y_train_pred_bag_svr))
print("Test data explained variance regression score:", explained_variance_score(original_y_test, y_test_pred_bag_svr))

In [ ]:
print("Train data R2 score:", r2_score(original_y_train, y_train_pred_bag_svr))
print("Test data R2 score:", r2_score(original_y_test, y_test_pred_bag_svr))

In [ ]:
trainPredictPlot, testPredictPlot = prepare_predictions_plot(close_price, y_train_pred_bag_svr, y_test_pred_bag_svr, 10)
plot_stock_prices(close_stock, close_price, trainPredictPlot, testPredictPlot)

## 3.3 Decision Tree Regressor

A Decision Tree Regressor can model complex relationships between input features and the target variable (in this case, stock prices) by recursively partitioning the data into subsets based on the values of the input features.

However, Decision Trees have some limitations when applied to time series data like stock prices. Decision Trees are not inherently designed to capture temporal dependencies present in time series data. To account for time dependencies, we will perform feature engineering that capture the historical information, such as lagged values or moving averages of the stock prices before building the model.



### 3.3.1 Feature Engineering

In this section, we will perform feature engineering to create additional features that capture relevant information from the time series data.

1. **Lagged values**: Create lagged features by shifting the close prices by different time steps (e.g., 1 day, 3 days). This captures the historical price information, which can help the model learn how past prices influence future prices.

2. **Moving averages**: Calculate simple moving averages (SMA) or exponential moving averages (EMA) of the close prices over different time windows (e.g., 5 days, 10 days, 30 days). Moving averages help smoothen the time series and capture trends in the data.

3. **Price change**: Compute the percentage change in close prices over different periods (e.g., daily, weekly, monthly). This feature can help the model capture the momentum and volatility of the stock price.

4. **Date-based features**: Extract features from the date, such as day of the week, day of the month, month, and quarter. These features can help the model capture any seasonality or cyclic patterns present in the stock price.

In [ ]:
df_dt = df[["Close", "Open", "High", "Low", "Volume"]]

In [ ]:
# Create lagged values
df_dt['lag_3'] = df_dt['Close'].shift(3)

# Calculate moving averages
df_dt['SMA_3'] = df_dt['Close'].rolling(window=3).mean()

# Calculate price change
df_dt['price_change'] = df_dt['Close'].diff()

# Add date-based features
df_dt['day_of_week'] = df['Date'].dt.dayofweek
df_dt['day_of_month'] = df['Date'].dt.day
df_dt['month'] = df['Date'].dt.month
df_dt['quarter'] = df['Date'].dt.quarter

# Remove rows with missing values caused by the shift and rolling operations
df_dt = df_dt.dropna()

Let's check the correlation between x variables.

In [ ]:
df_dt.corr()

However, we see that there are high correlations between features Open, High, and Low, so we decide to drop these 3 features.

In [ ]:
df_dt = df_dt.drop(["High", "Low", "Open"], axis =1)

### 3.3.2 Base Line Decision Tree Model

In [ ]:
X = df_dt.loc[:, df_dt.columns != "Close"]
y = df_dt['Close']
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train the Decision Tree Regressor
dt_regressor = DecisionTreeRegressor(random_state=42)
dt_regressor.fit(X_train, y_train)

In [ ]:
# Make predictions
y_train_pred_dt = dt_regressor.predict(X_train)
y_test_pred_dt = dt_regressor.predict(X_test)

#### 3.3.2.1 Evaluation (Base Line DT Model)

In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(y_train,y_train_pred_dt)))
print("Train data MSE: ", mean_squared_error(y_train,y_train_pred_dt))
print("Train data MAE: ", mean_absolute_error(y_train,y_train_pred_dt))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(y_test,y_test_pred_dt)))
print("Test data MSE: ", mean_squared_error(y_test,y_test_pred_dt))
print("Test data MAE: ", mean_absolute_error(y_test,y_test_pred_dt))

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(y_train, y_train_pred_dt))
print("Test data explained variance regression score:", explained_variance_score(y_test, y_test_pred_dt))

In [ ]:
print("Train data R2 score:", r2_score(y_train, y_train_pred_dt))
print("Test data R2 score:", r2_score(y_test, y_test_pred_dt))

The results of the Decision Tree Regressor model show a clear sign of overfitting. The training error metrics (RMSE, MSE, and MAE) are all 0, and the R2 score and explained variance regression score for the training data are both 1.0, which means the model has perfectly fit the training data. However, this perfection is not reflected in the test data results.

The test data error metrics (RMSE, MSE, and MAE) are significantly higher than those for the training data. Although the R2 score and explained variance regression score for the test data are relatively high (0.9526), the significant difference in performance between the training and test sets indicates that the model is overfitting to the training data and not generalizing well to new, unseen data.

To mitigate overfitting, we will consider the following approaches:

1. **Prune the decision tree**: Limit the depth of the tree, set a minimum number of samples required to split a node, or set a minimum number of samples required at a leaf node. These constraints can help make the tree less complex and more generalizable.

2. **Use cross-validation**: K-fold cross-validation can help you assess the model's performance more accurately and detect overfitting.

3. **Ensemble methods**: Combine multiple decision trees using methods like Random Forest or Gradient Boosting to reduce the likelihood of overfitting and improve the model's performance.

### 3.3.3 Decision Tree with Pruning

Pre-pruning is nothing but stopping the growth of the decision tree at an early stage. For that, we can limit the growth of trees by setting constraints. We can limit parameters like `max_depth`, `min_samples_split`, etc.

An effective way to do this is that we can grid search those parameters and choose the optimum values that give better performance on test data.

As of now, we will control these parameters

`max_depth`: maximum depth of decision tree

`min_sample_split`: The minimum number of samples required to split an internal node:

`min_samples_leaf`: The minimum number of samples required to be at a leaf node.

In [ ]:
params = {'max_depth': [4,8,12],
         'min_samples_split': [4,6,8,10],
         'min_samples_leaf': [3,5,7,9,11]}

clf = DecisionTreeRegressor(random_state=42)
gcv = GridSearchCV(estimator=clf,param_grid=params)
gcv.fit(X_train,y_train)

In [ ]:
gcv.best_estimator_

In [ ]:
model = gcv.best_estimator_
model.fit(X_train,y_train)
y_train_pred_dt_prune = model.predict(X_train)
y_test_pred_dt_prune = model.predict(X_test)
print("Depth: "+ str(model.tree_.max_depth)) #number of split levels
print("Leaves: "+ str(model.tree_.n_leaves))
fig = plt.figure(figsize=(50,45))
tree.plot_tree(model, feature_names = X_train.columns, filled=True, max_depth=2)
plt.show()

#### 3.3.3.1 Evaluation (Pruned DT Model)

In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(y_train,y_train_pred_dt_prune)))
print("Train data MSE: ", mean_squared_error(y_train,y_train_pred_dt_prune))
print("Train data MAE: ", mean_absolute_error(y_train,y_train_pred_dt_prune))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(y_test,y_test_pred_dt_prune)))
print("Test data MSE: ", mean_squared_error(y_test,y_test_pred_dt_prune))
print("Test data MAE: ", mean_absolute_error(y_test,y_test_pred_dt_prune))

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(y_train, y_train_pred_dt_prune))
print("Test data explained variance regression score:", explained_variance_score(y_test, y_test_pred_dt_prune))

In [ ]:
print("Train data R2 score:", r2_score(y_train, y_train_pred_dt_prune))
print("Test data R2 score:", r2_score(y_test, y_test_pred_dt_prune))

## 3.4 Random Forest Regressor

### 3.4.1 Base Line RFR

In [ ]:
rf = RandomForestRegressor(n_estimators = 500, max_depth = 50, random_state = 42)
rf.fit(X_train, y_train)

In [ ]:
y_train_pred_rf = rf.predict(X_train)
y_test_pred_rf = rf.predict(X_test)

#### 3.4.1.1 Evaluation (Base RFR)

In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(y_train,y_train_pred_rf)))
print("Train data MSE: ", mean_squared_error(y_train,y_train_pred_rf))
print("Train data MAE: ", mean_absolute_error(y_train,y_train_pred_rf))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(y_test,y_test_pred_rf)))
print("Test data MSE: ", mean_squared_error(y_test,y_test_pred_rf))
print("Test data MAE: ", mean_absolute_error(y_test,y_test_pred_rf))

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(y_train, y_train_pred_rf))
print("Test data explained variance regression score:", explained_variance_score(y_test, y_test_pred_rf))

In [ ]:
print("Train data R2 score:", r2_score(y_train, y_train_pred_rf))
print("Test data R2 score:", r2_score(y_test, y_test_pred_rf))

### 3.4.2 RFR Based on Time Series Data

In [ ]:
rf_ts = RandomForestRegressor(n_estimators = 500, max_depth = 50, random_state = 42)
rf_ts.fit(X_train_ts, y_train_ts)

In [ ]:
y_train_pred_rf_ts = rf_ts.predict(X_train_ts)
y_test_pred_rf_ts = rf_ts.predict(X_test_ts)

In [ ]:
# Transform back to original form
y_train_pred_rf_ts = scaler.inverse_transform(y_train_pred_rf_ts.reshape(-1,1))
y_test_pred_rf_ts = scaler.inverse_transform(y_test_pred_rf_ts.reshape(-1,1))
original_y_train = scaler.inverse_transform(y_train_ts.reshape(-1,1)) 
original_y_test = scaler.inverse_transform(y_test_ts.reshape(-1,1)) 

#### 3.4.2.1 Evaluation (Time Series RFR Model)

In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_y_train,y_train_pred_rf_ts)))
print("Train data MSE: ", mean_squared_error(original_y_train,y_train_pred_rf_ts))
print("Train data MAE: ", mean_absolute_error(original_y_train,y_train_pred_rf_ts))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_y_test,y_test_pred_rf_ts)))
print("Test data MSE: ", mean_squared_error(original_y_test,y_test_pred_rf_ts))
print("Test data MAE: ", mean_absolute_error(original_y_test,y_test_pred_rf_ts))

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_y_train, y_train_pred_rf_ts))
print("Test data explained variance regression score:", explained_variance_score(original_y_test, y_test_pred_rf_ts))

In [ ]:
print("Train data R2 score:", r2_score(original_y_train, y_train_pred_rf_ts))
print("Test data R2 score:", r2_score(original_y_test, y_test_pred_rf_ts))

## 3.5 LSTM

### 3.5.1 Baseline LSTM

In [ ]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(LSTM(32,return_sequences=True,input_shape=(time_step,1)))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
model.fit(X_train_ts,y_train_ts,validation_data=(X_train_ts,y_train_ts),epochs=200,batch_size=5,verbose=1)

In [ ]:
### Lets Do the prediction and check performance metrics
train_predict_lstm=model.predict(X_train_ts)
test_predict_lstm=model.predict(X_test_ts)
train_predict_lstm.shape, test_predict_lstm.shape

In [ ]:
# Transform back to original form
train_predict_lstm = scaler.inverse_transform(train_predict_lstm.reshape(-1,1))
test_predict_lstm = scaler.inverse_transform(test_predict_lstm.reshape(-1,1))
original_y_train = scaler.inverse_transform(y_train_ts.reshape(-1,1)) 
original_y_test = scaler.inverse_transform(y_test_ts.reshape(-1,1)) 

In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_y_train,train_predict_lstm)))
print("Train data MSE: ", mean_squared_error(original_y_train,train_predict_lstm))
print("Train data MAE: ", mean_absolute_error(original_y_train,train_predict_lstm))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_y_test,test_predict_lstm)))
print("Test data MSE: ", mean_squared_error(original_y_test,test_predict_lstm))
print("Test data MAE: ", mean_absolute_error(original_y_test,test_predict_lstm))

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_y_train, train_predict_lstm))
print("Test data explained variance regression score:", explained_variance_score(original_y_test, test_predict_lstm))

In [ ]:
print("Train data R2 score:", r2_score(original_y_train, train_predict_lstm))
print("Test data R2 score:", r2_score(original_y_test, test_predict_lstm))

In [ ]:
trainPredictPlot, testPredictPlot = prepare_predictions_plot(close_price, train_predict_lstm, test_predict_lstm, 10)
plot_stock_prices(close_stock, close_price, trainPredictPlot, testPredictPlot)

### 3.5.2 Hyperparameter Tuning LSTM

In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=512, step=32),
                   return_sequences=True,
                   input_shape=(X_train_ts.shape[1], 1)))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=512, step=32)))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=3)

In [ ]:
tuner.search(X_train_ts, y_train_ts, epochs=10, validation_data=(X_train_ts, y_train_ts))


In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = build_model(best_hps)
model.fit(X_train_ts, y_train_ts, epochs=200, validation_data=(X_train_ts, y_train_ts))


In [ ]:
### Lets Do the prediction and check performance metrics
train_predict_lstm_tuned=model.predict(X_train_ts)
test_predict_lstm_tuned=model.predict(X_test_ts)
train_predict_lstm_tuned.shape, test_predict_lstm_tuned.shape

In [ ]:
# Transform back to original form
train_predict_lstm_tuned = scaler.inverse_transform(train_predict_lstm_tuned.reshape(-1,1))
test_predict_lstm_tuned = scaler.inverse_transform(test_predict_lstm_tuned.reshape(-1,1))
original_y_train = scaler.inverse_transform(y_train_ts.reshape(-1,1)) 
original_y_test = scaler.inverse_transform(y_test_ts.reshape(-1,1)) 

In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_y_train,train_predict_lstm)))
print("Train data MSE: ", mean_squared_error(original_y_train,train_predict_lstm))
print("Train data MAE: ", mean_absolute_error(original_y_train,train_predict_lstm))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_y_test,test_predict_lstm)))
print("Test data MSE: ", mean_squared_error(original_y_test,test_predict_lstm))
print("Test data MAE: ", mean_absolute_error(original_y_test,test_predict_lstm))

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_y_train, train_predict_lstm_tuned))
print("Test data explained variance regression score:", explained_variance_score(original_y_test, test_predict_lstm_tuned))

In [ ]:
print("Train data R2 score:", r2_score(original_y_train, train_predict_lstm_tuned))
print("Test data R2 score:", r2_score(original_y_test, test_predict_lstm_tuned))

## 3.6 GRU

### 3.6.1 Baseline GRU

In [ ]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(GRU(32,return_sequences=True,input_shape=(time_step,1)))
model.add(GRU(32,return_sequences=True))
model.add(GRU(32,return_sequences=True))
model.add(GRU(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
model.fit(X_train_ts,y_train_ts,validation_data=(X_train_ts,y_train_ts),epochs=200,batch_size=5,verbose=1)

In [ ]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train_ts)
test_predict=model.predict(X_test_ts)
train_predict.shape, test_predict.shape

In [ ]:
# Transform back to original form
train_predict_gru = scaler.inverse_transform(train_predict.reshape(-1,1))
test_predict_gru = scaler.inverse_transform(test_predict.reshape(-1,1))
original_y_train = scaler.inverse_transform(y_train_ts.reshape(-1,1)) 
original_y_test = scaler.inverse_transform(y_test_ts.reshape(-1,1)) 

In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_y_train,train_predict_gru)))
print("Train data MSE: ", mean_squared_error(original_y_train,train_predict_gru))
print("Train data MAE: ", mean_absolute_error(original_y_train,train_predict_gru))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_y_test,test_predict_gru)))
print("Test data MSE: ", mean_squared_error(original_y_test,test_predict_gru))
print("Test data MAE: ", mean_absolute_error(original_y_test,test_predict_gru))

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_y_train, train_predict_gru))
print("Test data explained variance regression score:", explained_variance_score(original_y_test, test_predict_gru))

In [ ]:
print("Train data R2 score:", r2_score(original_y_train, train_predict_gru))
print("Test data R2 score:", r2_score(original_y_test, test_predict_gru))

### 3.6.2 Hyperparameter Tuning GRU

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
tf.keras.backend.clear_session()
# define the GRU model architecture
def build_model(units=32):
    model = Sequential()
    model.add(GRU(units=units, return_sequences=True, input_shape=(X_train_ts.shape[1], 1)))
    model.add(GRU(units=units))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# create a KerasRegressor wrapper for the model
keras_reg = KerasRegressor(build_fn=build_model, verbose=0)

# define the hyperparameters to search over
param_dist = {
    'units': sp_randint(32, 512),
    'batch_size': sp_randint(32, 128),
    'epochs': sp_randint(50, 200)
}

# perform Random Search using cross-validation to find the best hyperparameters
search = RandomizedSearchCV(keras_reg, param_distributions=param_dist, n_iter=5, cv=3, error_score='raise')
search_results = search.fit(X_train_ts, y_train_ts)

# get the best hyperparameters and fit the model with them
best_params = search_results.best_params_
model = build_model(units=best_params['units'])
model.fit(X_train_ts, y_train_ts, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=0)

# evaluate the model on the test set
score = model.evaluate(X_test_ts, y_test_ts, verbose=0)
print('Test loss:', score)

In [ ]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train_ts)
test_predict=model.predict(X_test_ts)
train_predict.shape, test_predict.shape

In [ ]:
# Transform back to original form
train_predict_gru_tuned = scaler.inverse_transform(train_predict.reshape(-1,1))
test_predict_gru_tuned = scaler.inverse_transform(test_predict.reshape(-1,1))
original_y_train = scaler.inverse_transform(y_train_ts.reshape(-1,1)) 
original_y_test = scaler.inverse_transform(y_test_ts.reshape(-1,1))

In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_y_train,train_predict_gru_tuned)))
print("Train data MSE: ", mean_squared_error(original_y_train,train_predict_gru_tuned))
print("Train data MAE: ", mean_absolute_error(original_y_train,train_predict_gru_tuned))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_y_test,test_predict_gru_tuned)))
print("Test data MSE: ", mean_squared_error(original_y_test,test_predict_gru_tuned))
print("Test data MAE: ", mean_absolute_error(original_y_test,test_predict_gru_tuned))

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(original_y_train, train_predict_gru_tuned))
print("Test data explained variance regression score:", explained_variance_score(original_y_test, test_predict_gru_tuned))

In [ ]:
print("Train data R2 score:", r2_score(original_y_train, train_predict_gru_tuned))
print("Test data R2 score:", r2_score(original_y_test, test_predict_gru_tuned))

## 3.2 ARIMA

In [ ]:
df_arima = df[["Date", "Close"]]
df_arima.set_index(['Date'],inplace=True)
df_arima.head()

### 3.2.1 Plotting the data

In [ ]:
df_arima.plot(figsize=(20,5))
plt.ylabel('Close')
plt.title('Close Price')
plt.show()

### 3.2.2 Checking for stationarity using Augmented Dickey Fuller Test

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adfuller_test(trends):
    result = adfuller(trends)
    labels = ['ADF Test Statistic','p-value','#Lags Used','#Observation Used']
    for value,label in zip(result,labels):
        print(label  + ': ' + str(value))
    if result[1]<=0.05:
        print('Strong evidence against the null hypothesis, Hence REJECT Ho. and The series is Stationary')
    else:
        print('week evidence against null hypothesis, Hence ACCEPT Ho. that the series is not stationary.')
        
adfuller_test(df_arima['Close'])

### 3.2.3 Differencing Data ==> Make it stationary

In [ ]:
diff1=df_arima-df_arima.shift(1)
diff1=diff1.dropna()
adfuller_test(diff1)
diff1.plot(figsize=(20,5))

The data is now stationary and the order of differencing is 1.

### 3.2.4 Plotting Autocorrelation and Partial Autocorreltion functions

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sgt.plot_acf(diff1['Close'], ax=ax, lags=30)
plt.show()

### 3.2.5 Model Implementation

In [ ]:

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# Use the 'Close' column as the target variable
price = df['Close']
# price=price-price.shift(1)
# price=price.dropna()

# Split the data into training and testing sets
train_size = int(len(price) * 0.8)
train, test = price[:train_size], price[train_size:]

# Fit the ARIMA model
p, d, q = 3,1,5
model = ARIMA(train, order=(p, d, q))
model_fit = model.fit()

# Make predictions
predictions = model_fit.forecast(steps=len(test))
predictions

# Plot the actual and predicted stock prices
plt.figure(figsize=(12, 6))
plt.plot(test.index, test)
plt.plot(predictions)
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.legend(['Actual', 'Prediction'])

In [ ]:
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(test, predictions)))
print("Test data MSE: ", mean_squared_error(test, predictions))
print("Test data MAE: ", mean_absolute_error(test, predictions))

In [ ]:
print("Test data explained variance regression score:", explained_variance_score(test, predictions))

In [ ]:
print("Test data R2 score:", r2_score(test, predictions))

In [ ]:
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_y_train,y_train_pred_rf_ts)))